In [1]:
import os
from tqdm import tqdm

import numpy as np
import pandas as pd


SEED = 42
plt.style.use("ggplot")
pd.options.display.max_columns = None

## 1. Data

In [2]:
path_train = "data/train_test_dataset_Fault Impact Analysis/"
path_test = "data/validation_clean/"
ss_df = pd.read_csv("data/SampleSubmission.csv")

In [3]:
def make_df(path):
    df = pd.DataFrame()
    files = [elt for elt in os.listdir(path) if os.path.isfile(os.path.join(path, elt))]
    
    for file in tqdm(files):
        name = file.split(".")[0] # Name NE 
        idx = name.split("_")[-1]  # ID NE
        if "train" in path:
            data = pd.read_csv(path+file)
        else:
            data = pd.read_csv(path+file, index_col=0)
        
        data = data.rename(columns={"NE ID":"NE"})
        data["ID"] = idx
        df = pd.concat((df, data))

    return df

In [4]:
print("Build train dataframe")
train_df = make_df(path_train)

print("Build test dataframe")
test_df = make_df(path_test)

Build train dataframe


100%|███████████████████████████████████████| 7256/7256 [12:23<00:00,  9.75it/s]


Build test dataframe


100%|███████████████████████████████████████| 1932/1932 [01:14<00:00, 25.99it/s]


In [5]:
train_df["NE_ID"] = train_df["NE"].astype(str) + "_" + train_df["ID"].astype(str)
test_df["NE_ID"] = test_df["NE"].astype(str) + "_" + test_df["ID"].astype(str)

In [6]:
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [7]:
print(f"Train Shape: {train_df.shape}\nTest Shape : {test_df.shape}")

Train Shape: (908922, 13)
Test Shape : (175150, 13)


In [8]:
def create_target(df):
    tmp = df.groupby('NE_ID')['data_rate'].\
             rolling(window=2).\
             apply(lambda x: x.iloc[1] - x.iloc[0]).\
             fillna(0)
    tmp = tmp.to_frame().rename({"data_rate":"diff"}, axis=1)
    
    df = df.reset_index().merge(tmp, left_on=["NE_ID", "index"], right_index=True)
    df["marker1"] = df["diff"]<0
    df["marker2"] = df["fault_duration"]>=1
    df["target"] = (df["marker1"] & df["marker2"]).astype(int)
    
    df = df.drop(["diff", "marker1", "marker2"], axis=1)

    return df

In [9]:
%%time
train_df = create_target(train_df.copy())

CPU times: user 3min 25s, sys: 498 ms, total: 3min 26s
Wall time: 3min 26s


In [10]:
print(f"Train Shape: {train_df.shape}")

Train Shape: (908922, 15)


In [11]:
print(f"Values Count \"Target\":\n{train_df.target.value_counts()}")

Values Count "Target":
target
0    863207
1     45715
Name: count, dtype: int64


In [12]:
860207 + 45715

905922

In [13]:
# For each NE_ID, delete timesteps which come after the first 1
def remove_duplicate(df):
    tmp = pd.DataFrame()
    
    ne_ids_1 = set(df[df.target==1]["NE_ID"].unique())
    ne_ids_0 = set(df["NE_ID"].unique()).difference(ne_ids_1)
    
    # Save NE which only have "0"
    tmp = df[df.NE_ID.isin(list(ne_ids_0))]
    
    # First occurences of "1"
    first_occ = df[df.NE_ID.isin(list(ne_ids_1))].groupby("NE_ID")['target'].idxmax()
    
    for ne_id in tqdm(first_occ.index):
        tmp = pd.concat((tmp, df[(df.NE_ID==ne_id) & (df["index"]<=first_occ.loc[ne_id])] ))
    
    tmp = tmp.drop(["index"], axis=1).reset_index(drop=True)
        
    return tmp

In [14]:
train_df = remove_duplicate(train_df.copy())

100%|███████████████████████████████████████| 4733/4733 [26:55<00:00,  2.93it/s]


In [15]:
print(f"Values Count \"Target\":\n{train_df.target.value_counts()}")

Values Count "Target":
target
0    818320
1      4733
Name: count, dtype: int64


In [16]:
print(f"Number of NEs: {train_df.NE_ID.unique().shape[0]}")

Number of NEs: 7256


In [17]:
train_df.isna().sum()

NE                            0
endTime                       0
access_success_rate        5439
resource_utilition_rate    5439
TA                         5439
bler                       5439
cqi                        5439
mcs                        5439
data_rate                     0
fault_duration                0
relation                      0
ID                            0
NE_ID                         0
target                        0
dtype: int64

In [18]:
test_df.isna().sum()

NE                             0
endTime                        0
access_success_rate        50869
resource_utilition_rate     2030
TA                          2722
bler                        2419
cqi                         2359
mcs                         2360
data_rate                   2905
fault_duration                 0
relation                       0
ID                             0
NE_ID                          0
dtype: int64

### Fillna in train

In [19]:
# Fill "TA"
ne_ids = train_df[train_df.TA.isna()]["NE_ID"].unique()
for ne_id in tqdm(ne_ids):
    values = train_df[train_df.NE_ID==ne_id]["TA"].interpolate(method="spline", order=2, limit_direction="both")
    train_df.loc[train_df.NE_ID==ne_id, "TA"] = values

train_df["TA"] = train_df["TA"].fillna(0)

100%|█████████████████████████████████████████| 489/489 [03:08<00:00,  2.60it/s]


In [20]:
# Fill "bler"
ne_ids = train_df[train_df.bler.isna()]["NE_ID"].unique()
for ne_id in tqdm(ne_ids):    
    values = train_df[train_df.NE_ID==ne_id]["bler"].interpolate(method="spline", order=2, limit_direction="both")
    train_df.loc[train_df.NE_ID==ne_id, "bler"] = values
train_df["bler"] = train_df["bler"].fillna(0)

100%|█████████████████████████████████████████| 489/489 [03:11<00:00,  2.55it/s]


In [21]:
# Fill "cqi"
ne_ids = train_df[train_df.cqi.isna()]["NE_ID"].unique()
for ne_id in tqdm(ne_ids):
    values = train_df[train_df.NE_ID==ne_id]["cqi"].interpolate(method="spline", order=2, limit_direction="both")
    train_df.loc[train_df.NE_ID==ne_id, "cqi"] = values
train_df["cqi"] = train_df["cqi"].fillna(0)

100%|█████████████████████████████████████████| 489/489 [03:06<00:00,  2.62it/s]


In [22]:
# Fill "mcs"
ne_ids = train_df[train_df.mcs.isna()]["NE_ID"].unique()
for ne_id in tqdm(ne_ids):
    values = train_df[train_df.NE_ID==ne_id]["mcs"].interpolate(method="spline", order=2, limit_direction="both")
    train_df.loc[train_df.NE_ID==ne_id, "mcs"] = values
train_df["mcs"] = train_df["mcs"].fillna(0)

100%|█████████████████████████████████████████| 489/489 [03:11<00:00,  2.55it/s]


In [23]:
# Fill "resource_utilition_rate"
ne_ids = train_df[train_df.resource_utilition_rate.isna()]["NE_ID"].unique()
for ne_id in tqdm(ne_ids):
    values = train_df[train_df.NE_ID==ne_id]["resource_utilition_rate"].interpolate(method="spline", order=2, limit_direction="both")
    train_df.loc[train_df.NE_ID==ne_id, "resource_utilition_rate"] = values
train_df["resource_utilition_rate"] = train_df["resource_utilition_rate"].fillna(0)

100%|█████████████████████████████████████████| 489/489 [03:15<00:00,  2.50it/s]


In [24]:
# Mask last time step of each NEs
tmp = train_df.groupby("NE_ID").max()['endTime'].to_frame()
train_df = train_df.merge(tmp, on=["NE_ID", "endTime"], how="left", indicator=True)
cols = ["access_success_rate", "resource_utilition_rate", "data_rate", "fault_duration", "relation", "TA", "bler", "cqi", "mcs"]
train_df.loc[train_df._merge=="both", cols] = -1
train_df = train_df.drop(["_merge"], axis=1)

### Fillna in test

In [25]:
# Mask last time step of each NEs
tmp = test_df.groupby("NE_ID").max()['endTime'].to_frame()
test_df = test_df.merge(tmp, on=["NE_ID", "endTime"], how="left", indicator=True)
cols = ["access_success_rate", "resource_utilition_rate", "data_rate", "fault_duration", "relation", "TA", "bler", "cqi", "mcs"]
test_df.loc[test_df._merge=="both", cols] = -1
test_df = test_df.drop(["_merge"], axis=1)

In [26]:
tmp = (test_df.TA.isna() & test_df.bler.isna() & test_df.cqi.isna() & test_df.mcs.isna())
index = tmp[tmp.values == True].index
test_df = test_df.drop(index, axis=0)

In [27]:
# Fill na "bler"
ne_ids = test_df[test_df.bler.isna()]["NE_ID"].unique()
for ne_id in ne_ids:
    values = test_df[test_df.NE_ID==ne_id]["bler"].interpolate(method="spline", order=2)
    test_df.loc[test_df.NE_ID==ne_id, "bler"] = values
# test_df.loc[test_df.bler.isna(), "bler"] = 0

# fill na "mcs"
ne_ids = test_df[test_df.mcs.isna()]["NE_ID"].unique()
for ne_id in ne_ids:
    values = test_df[test_df.NE_ID==ne_id]["mcs"].interpolate(method="spline", order=2)
    test_df.loc[test_df.NE_ID==ne_id, "mcs"] = values

# fill na "TA"
ne_ids = test_df[test_df.TA.isna()]["NE_ID"].unique()
for ne_id in ne_ids:
    if ne_id == "B0108-25_3":
        test_df.loc[test_df.NE_ID==ne_id, "TA"] = 1
    else:
        values = test_df[test_df.NE_ID==ne_id]["TA"].interpolate(method="spline", order=3, limit_direction="both")
        test_df.loc[test_df.NE_ID==ne_id, "TA"] = values

In [28]:
# Fill "resource_utilition_rate"
ne_ids = test_df[test_df.resource_utilition_rate.isna()]["NE_ID"].unique()
for ne_id in tqdm(ne_ids):
    values = test_df[test_df.NE_ID==ne_id]["resource_utilition_rate"].interpolate(method="spline", order=2, limit_direction="both")
    test_df.loc[test_df.NE_ID==ne_id, "resource_utilition_rate"] = values
test_df["resource_utilition_rate"] = test_df["resource_utilition_rate"].fillna(0)

100%|███████████████████████████████████████████| 21/21 [00:02<00:00, 10.44it/s]


In [29]:
# Fill "data_rate"
ne_ids = test_df[test_df.data_rate.isna()]["NE_ID"].unique()
for ne_id in tqdm(ne_ids):
    values = test_df[test_df.NE_ID==ne_id]["data_rate"].interpolate(method="spline", order=2, limit_direction="both")
    test_df.loc[test_df.NE_ID==ne_id, "data_rate"] = values
test_df["data_rate"] = test_df["data_rate"].fillna(0)

100%|███████████████████████████████████████████| 22/22 [00:02<00:00,  8.19it/s]


In [30]:
train_df.isna().sum()

NE                            0
endTime                       0
access_success_rate        5262
resource_utilition_rate       0
TA                            0
bler                          0
cqi                           0
mcs                           0
data_rate                     0
fault_duration                0
relation                      0
ID                            0
NE_ID                         0
target                        0
dtype: int64

In [31]:
test_df.isna().sum()

NE                             0
endTime                        0
access_success_rate        48510
resource_utilition_rate        0
TA                             0
bler                           0
cqi                            0
mcs                            0
data_rate                      0
fault_duration                 0
relation                       0
ID                             0
NE_ID                          0
dtype: int64

In [32]:
to_dels = ["access_success_rate"]
test_df = test_df.drop(to_dels, axis=1)
train_df = train_df.drop(to_dels, axis=1)

### Make Dataframes Persistent

In [33]:
train_df.to_csv("data/train.csv", index=False)
test_df.to_csv("data/test.csv", index=False)

In [34]:
train_df.head()

,NE,endTime,resource_utilition_rate,TA,bler,cqi,mcs,data_rate,fault_duration,relation,ID,NE_ID,target
0,B0010-72,2023-02-09 01:00:00,1.025,3.389423,10.727969,7.492512,3.002243,17.988247,0.0,0.0,1,B0010-72_1,0
1,B0010-72,2023-02-09 02:00:00,0.854,3.006494,8.157557,7.882693,1.758560,12.360708,0.0,0.0,1,B0010-72_1,0
2,B0010-72,2023-02-09 03:00:00,0.819,3.307018,17.902962,7.672714,1.140774,10.075042,0.0,0.0,1,B0010-72_1,0
3,B0010-72,2023-02-09 04:00:00,0.823,2.185185,8.175228,7.193010,2.903567,21.281706,0.0,0.0,1,B0010-72_1,0
4,B0010-72,2023-02-09 05:00:00,0.893,2.350515,7.645895,8.975481,5.679832,16.526131,0.0,0.0,1,B0010-72_1,0


In [35]:
test_df.head()

,NE,endTime,resource_utilition_rate,TA,bler,cqi,mcs,data_rate,fault_duration,relation,ID,NE_ID
0,B0017-32,2023-02-19 11:00:00,60.941,2.727843,12.841164,6.161731,6.602028,3.161234,0.0,0.0,10,B0017-32_10
1,B0017-32,2023-02-19 12:00:00,-1.000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,10,B0017-32_10
2,B0073-45,2023-02-19 13:00:00,8.294,2.357810,9.107972,7.166678,6.187408,6.932266,0.0,0.0,24,B0073-45_24
3,B0073-45,2023-02-19 14:00:00,-1.000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,24,B0073-45_24
4,B0093-23,2023-02-19 19:00:00,56.760,1.037665,11.913121,8.925502,11.309321,25.287751,0.0,0.0,21,B0093-23_21


In [36]:
ss_df

,ID,data_rate_t+1_trend
0,B0017-25_24,0
1,B0017-25_25,0
2,B0017-25_26,0
3,B0017-25_27,0
4,B0017-32_1,0
...,...,...
1927,B0122-43_1,0
1928,B0122-43_2,0
1929,B0122-45_1,0
1930,B0122-45_2,0


In [37]:
train_df.shape, test_df.shape

((823053, 13), (174723, 12))